In [9]:
import aopy 
#aopy as in Orsborn lab's data processing pipeline, 
#can be downloaded here https://github.com/aolabNeuro/analyze
import numpy as np
import matplotlib.pyplot as plt
import seaborn 
import scipy
from scipy.optimize import minimize,least_squares
import copy as copy
import time as tpy
import glob
import os
import pandas as pd
import pickle

seaborn.set()

# CPHS data extraction
## extract data and save to pandas dataframe

- 10 participants (METAS103-METAS113)
- 2 blocks (BLOCK1, BLOCK2)
- 
- `weiner_task_data` $\rightarrow$ data that we want
- `quattro_data` $\rightarrow$ pretty sure this is raw EMG data

ISSUE: can't analyze block 1 AND block 2 -- memory issue

In [10]:
# weiner_task_data_METACPHS_S102_L2_a_25_D_1_BLOCK_0_pE_1e-06_pD_1e-03_pF_1e-07.h5
# weiner_task_data_METACPHS_S102_L2_a_25_D_6_BLOCK_0_pE_1e-06_pD_1e-04_pF_1e-07.h5
# weiner_task_data_METACPHS_S102_L2_a_50_D_1_BLOCK_0_pE_1e-06_pD_1e-03_pF_1e-07.h5
# weiner_task_data_METACPHS_S102_L2_a_50_D_2_BLOCK_0_pE_1e-06_pD_1e-04_pF_1e-07.h5
# weiner_task_data_METACPHS_S102_L2_a_50_D_5_BLOCK_0_pE_1e-06_pD_1e-03_pF_1e-07.h5
# weiner_task_data_METACPHS_S102_L2_a_50_D_6_BLOCK_0_pE_1e-06_pD_1e-04_pF_1e-07.h5
# weiner_task_data_METACPHS_S102_L2_a_75_D_3_BLOCK_0_pE_1e-06_pD_1e-03_pF_1e-07.h5
# weiner_task_data_METACPHS_S102_L2_a_75_D_4_BLOCK_0_pE_1e-06_pD_1e-04_pF_1e-07.h5
# weiner_task_data_METACPHS_S102_L2_a_75_D_7_BLOCK_0_pE_1e-06_pD_1e-03_pF_1e-07.h5
# weiner_task_data_METACPHS_S102_L2_a_75_D_8_BLOCK_0_pE_1e-06_pD_1e-04_pF_1e-07.h5

In [11]:
keys = ['METACPHS_S106', 
        'METACPHS_S107',
        'METACPHS_S108', 
        'METACPHS_S109', 
        'METACPHS_S110', 
        'METACPHS_S111', 
        'METACPHS_S112',
        'METACPHS_S113',
        'METACPHS_S114',
        'METACPHS_S115',
        'METACPHS_S116',
        'METACPHS_S117',
        'METACPHS_S118',
        'METACPHS_S119']

# create dataframe
var_names = ['refs','poss','dec_vels','int_vels','emgs','Ws','Hs','alphas', 'pDs','times','conditions']

## BLOCK 1
refs_block1 = {key: [] for key in keys} # re
poss_block1 = {key: [] for key in keys}
dec_vels_block1 = {key: [] for key in keys}                                            
emgs_block1 = {key: [] for key in keys}
int_vel_block1 = {key: [] for key in keys}
Ws_block1 = {key: [] for key in keys}
Hs_block1 = {key: [] for key in keys}
alphas_block1 = {key: [] for key in keys}
pDs_block1 = {key: [] for key in keys}
times_block1 = {key: [] for key in keys}
conditions_block1 = {key: [] for key in keys}

## BLOCK 2
refs_block2 = {key: [] for key in keys} # re
poss_block2 = {key: [] for key in keys}
dec_vels_block2 = {key: [] for key in keys}
emgs_block2 = {key: [] for key in keys}
int_vel_block2 = {key: [] for key in keys}
Ws_block2 = {key: [] for key in keys}
Hs_block2 = {key: [] for key in keys}
alphas_block2 = {key: [] for key in keys}
pDs_block2 = {key: [] for key in keys}
times_block2 = {key: [] for key in keys}
conditions_block2 = {key: [] for key in keys}
# decoders_block1 = {key: [] for key in keys}

In [12]:
keys

['METACPHS_S106',
 'METACPHS_S107',
 'METACPHS_S108',
 'METACPHS_S109',
 'METACPHS_S110',
 'METACPHS_S111',
 'METACPHS_S112',
 'METACPHS_S113',
 'METACPHS_S114',
 'METACPHS_S115',
 'METACPHS_S116',
 'METACPHS_S117',
 'METACPHS_S118',
 'METACPHS_S119']

In [13]:
print('analyze block 1')
for key in keys:
    print('analyzing', key)
    PATH = 'F:/cphs/'
#     PATH = '/Volumes/NO NAME/'
    print(PATH)
    data_dir = PATH + key + '/BLOCK1'
    print(data_dir)
    for name in glob.glob(data_dir + '/weiner_task_data*.h5'):
        file_name = os.path.basename(name)
        print(file_name)
        
        weiner_task_data, weiner_config = aopy.data.load_bmi3d_hdf_table(data_dir, file_name, 'weiner')
        f = weiner_task_data
        c = weiner_config
#         print(f.dtype.names) # get the names of the fields
        refs_block1[key].append(np.asarray(f['reference']))
        poss_block1[key].append(np.asarray(f['decoded_position']))
        dec_vels_block1[key].append(np.asarray(f['decoded_velocity']))
        int_vel_block1[key].append(np.asarray(f['intended_velocity'])) # didn't seem to be saving
        emgs_block1[key].append(np.asarray(f['filtered_emg']))
        Ws_block1[key].append(np.asarray(f['weiner_filter_w']))
        Hs_block1[key].append(np.asarray(f['weiner_filter_h'][0]))
        alphas_block1[key].append(c['alpha'])
        pDs_block1[key].append(c['lambdaD'])
        time = np.asarray(f['timestamp']).flatten()
        times_block1[key].append(time - time[0])
        conditions_block1[key].append(file_name[-41:-38])
    refs_block1[key] = np.asarray(refs_block1[key])
    poss_block1[key] = np.asarray(poss_block1[key])
    dec_vels_block1[key] = np.asarray(dec_vels_block1[key])
    emgs_block1[key] = np.asarray(emgs_block1[key])
    Ws_block1[key] = np.asarray(Ws_block1[key])
    Hs_block1[key] = np.asarray(Hs_block1[key])
    alphas_block1[key] = np.asarray(alphas_block1[key])
    pDs_block1[key] = np.asarray(pDs_block1[key])
    times_block1[key] = np.asarray(times_block1[key])
    conditions_block1[key] = np.asarray(conditions_block1[key])
    int_vel_block1[key] = np.asarray(int_vel_block1[key])

analyze block 1
analyzing METACPHS_S106
F:/cphs/
F:/cphs/METACPHS_S106/BLOCK1
weiner_task_data_METACPHS_S106_L2_a_25_D_1_BLOCK_1_pE_1e-06_pD_1e-03_pF_1e-07.h5
weiner_task_data_METACPHS_S106_L2_a_25_D_2_BLOCK_1_pE_1e-06_pD_1e-04_pF_1e-07.h5
weiner_task_data_METACPHS_S106_L2_a_25_D_5_BLOCK_1_pE_1e-06_pD_1e-03_pF_1e-07.h5
weiner_task_data_METACPHS_S106_L2_a_25_D_6_BLOCK_1_pE_1e-06_pD_1e-04_pF_1e-07.h5
weiner_task_data_METACPHS_S106_L2_a_75_D_3_BLOCK_1_pE_1e-06_pD_1e-03_pF_1e-07.h5
weiner_task_data_METACPHS_S106_L2_a_75_D_4_BLOCK_1_pE_1e-06_pD_1e-04_pF_1e-07.h5
weiner_task_data_METACPHS_S106_L2_a_75_D_7_BLOCK_1_pE_1e-06_pD_1e-03_pF_1e-07.h5
weiner_task_data_METACPHS_S106_L2_a_75_D_8_BLOCK_1_pE_1e-06_pD_1e-04_pF_1e-07.h5
analyzing METACPHS_S107
F:/cphs/
F:/cphs/METACPHS_S107/BLOCK1
weiner_task_data_METACPHS_S107_L2_a_25_D_1_BLOCK_1_pE_1e-06_pD_1e-03_pF_1e-07.h5
weiner_task_data_METACPHS_S107_L2_a_25_D_2_BLOCK_1_pE_1e-06_pD_1e-04_pF_1e-07.h5
weiner_task_data_METACPHS_S107_L2_a_25_D_5_BLOCK_1

In [14]:
print('analyze block 2')
for key in keys:
    print('analyzing', key)
    PATH =  'F:/cphs/'
    print(PATH)
    data_dir = PATH + key +'/BLOCK2'
    for name in glob.glob(data_dir + '/weiner_task_data*.h5'):
        file_name = os.path.basename(name)
        print(file_name)
        
        weiner_task_data, weiner_config = aopy.data.load_bmi3d_hdf_table(data_dir, file_name, 'weiner')
        f = weiner_task_data
        c = weiner_config
#         print(f.dtype.names) # get the names of the fields
        refs_block2[key].append(np.asarray(f['reference']))
        poss_block2[key].append(np.asarray(f['decoded_position']))
        dec_vels_block2[key].append(np.asarray(f['decoded_velocity']))
        int_vel_block2[key].append(np.asarray(f['intended_velocity'])) # didn't seem to be saving
        emgs_block2[key].append(np.asarray(f['filtered_emg']))
        Ws_block2[key].append(np.asarray(f['weiner_filter_w']))
        Hs_block2[key].append(np.asarray(f['weiner_filter_h'][0]))
        alphas_block2[key].append(c['alpha'])
        pDs_block2[key].append(c['lambdaD'])
        time = np.asarray(f['timestamp']).flatten()
        times_block2[key].append(time - time[0])
        conditions_block2[key].append(file_name[-41:-38])
    refs_block2[key] = np.asarray(refs_block2[key])
    poss_block2[key] = np.asarray(poss_block2[key])
    dec_vels_block2[key] = np.asarray(dec_vels_block2[key])
    emgs_block2[key] = np.asarray(emgs_block2[key])
    Ws_block2[key] = np.asarray(Ws_block2[key])
    Hs_block2[key] = np.asarray(Hs_block2[key])
    alphas_block2[key] = np.asarray(alphas_block2[key])
    pDs_block2[key] = np.asarray(pDs_block2[key])
    times_block2[key] = np.asarray(times_block2[key])
    conditions_block2[key] = np.asarray(conditions_block2[key])
    int_vel_block2[key] = np.asarray(int_vel_block2[key])

analyze block 2
analyzing METACPHS_S106
F:/cphs/
weiner_task_data_METACPHS_S106_L2_a_25_D_1_BLOCK_2_pE_1e-06_pD_1e-03_pF_1e-07.h5
weiner_task_data_METACPHS_S106_L2_a_25_D_2_BLOCK_2_pE_1e-06_pD_1e-04_pF_1e-07.h5
weiner_task_data_METACPHS_S106_L2_a_25_D_5_BLOCK_2_pE_1e-06_pD_1e-03_pF_1e-07.h5
weiner_task_data_METACPHS_S106_L2_a_25_D_6_BLOCK_2_pE_1e-06_pD_1e-04_pF_1e-07.h5
weiner_task_data_METACPHS_S106_L2_a_75_D_3_BLOCK_2_pE_1e-06_pD_1e-03_pF_1e-07.h5
weiner_task_data_METACPHS_S106_L2_a_75_D_4_BLOCK_2_pE_1e-06_pD_1e-04_pF_1e-07.h5
weiner_task_data_METACPHS_S106_L2_a_75_D_7_BLOCK_2_pE_1e-06_pD_1e-03_pF_1e-07.h5
weiner_task_data_METACPHS_S106_L2_a_75_D_8_BLOCK_2_pE_1e-06_pD_1e-04_pF_1e-07.h5
analyzing METACPHS_S107
F:/cphs/
weiner_task_data_METACPHS_S107_L2_a_25_D_1_BLOCK_2_pE_1e-06_pD_1e-03_pF_1e-07.h5
weiner_task_data_METACPHS_S107_L2_a_25_D_2_BLOCK_2_pE_1e-06_pD_1e-04_pF_1e-07.h5
weiner_task_data_METACPHS_S107_L2_a_25_D_5_BLOCK_2_pE_1e-06_pD_1e-03_pF_1e-07.h5
weiner_task_data_METACPHS_S

In [15]:
PATH

'F:/cphs/'

In [16]:
data = [refs_block1, poss_block1, dec_vels_block1, int_vel_block1, emgs_block1, Ws_block1,Hs_block1,alphas_block1,pDs_block1,times_block1,conditions_block1]

with open(PATH + 'continuous_full_data_block1.pickle','wb') as handle:
    pickle.dump(data,handle,protocol=pickle.HIGHEST_PROTOCOL)

In [ ]:
data = [refs_block2, poss_block2, dec_vels_block2, int_vel_block2, emgs_block2, Ws_block2, Hs_block2, alphas_block2, pDs_block2, times_block2, conditions_block2]

with open(PATH + 'continuous_full_data_block2.pickle','wb') as handle:
    pickle.dump(data,handle,protocol=pickle.HIGHEST_PROTOCOL)

# extract data and save to pandas dataframe

- 7 participants (METAS003-METAS009)
- 2 blocks (BLOCK1, BLOCK2)
- 10 minute trial of 1 decoder for BLOCK3
- 6 conditions 1-3 are for positive initialization, 4-6 are for negative initialization (`a_0.25_d_1`, `a_0.5_d_2`, `a_0.75_d_3`, `a_0.25_d_4`, `a_0.5_d_5`, `a_0.75_d_6`)
- `weiner_task_data` $\rightarrow$ data that we want
- `quattro_data` $\rightarrow$ pretty sure this is raw EMG data

ISSUE: can't analyze block 1 AND block 2 -- memory issue

In [ ]:
keys = ['METAS100', 'METAS101']

# create dataframe
var_names = ['refs','poss','dec_vels','int_vels','emgs','Ws','Hs','alphas','times','conditions']

# refs_block1 = {key: [] for key in keys}
# poss_block1 = {key: [] for key in keys}
# dec_vels_block1 = {key: [] for key in keys}
# emgs_block1 = {key: [] for key in keys}
# int_vel_block1 = {key: [] for key in keys}
# Ws_block1 = {key: [] for key in keys}
# Hs_block1 = {key: [] for key in keys}
# alphas_block1 = {key: [] for key in keys}
# times_block1 = {key: [] for key in keys}
# conditions_block1 = {key: [] for key in keys}

# refs_block2 = {key: [] for key in keys}
# poss_block2 = {key: [] for key in keys}
# dec_vels_block2 = {key: [] for key in keys}
# emgs_block2 = {key: [] for key in keys}
# int_vel_block2 = {key: [] for key in keys}
# Ws_block2 = {key: [] for key in keys}
# Hs_block2 = {key: [] for key in keys}
# alphas_block2 = {key: [] for key in keys}
# times_block2 = {key: [] for key in keys}
# conditions_block2 = {key: [] for key in keys}

refs_block = {key: [] for key in keys}
poss_block = {key: [] for key in keys}
dec_vels_block = {key: [] for key in keys}
emgs_block = {key: [] for key in keys}
int_vel_block = {key: [] for key in keys}
Ws_block = {key: [] for key in keys}
Hs_block = {key: [] for key in keys}
alphas_block = {key: [] for key in keys}
times_block = {key: [] for key in keys}
conditions_block = {key: [] for key in keys}

In [ ]:
# GET BLOCK 1
print('analyze block 1')
for key in keys:
    print('analyzing', key)
    PATH = 'data/' + key + '/BLOCK1/weiner_task_data*'
    print(PATH)
    data_dir = 'data/' + key + '/BLOCK1/'
    for name in glob.glob(PATH):
        file_name = os.path.basename(name)
        print(file_name)

#         weiner_task_data, weiner_config = aopy.data.load_bmi3d_hdf_table(data_dir, file_name, 'weiner')
#         f = weiner_task_data
# #         print(f.dtype.names) # get the names of the fields
#         refs_block1[key].append(np.asarray(f['reference']))
#         poss_block1[key].append(np.asarray(f['decoded_position']))
#         dec_vels_block1[key].append(np.asarray(f['decoded_velocity']))
#         int_vel_block1[key].append(np.asarray(f['intended_velocity'])) # didn't seem to be saving
#         emgs_block1[key].append(np.asarray(f['filtered_emg']))
#         Ws_block1[key].append(np.asarray(f['weiner_filter_w']))
#         Hs_block1[key].append(np.asarray(f['weiner_filter_h'][0]))
#         alphas_block1[key].append(weiner_config['alpha'])
#         time = np.asarray(f['timestamp']).flatten()
#         times_block1[key].append(time - time[0])
#         conditions_block1[key].append(file_name[-13:-3])
#     refs_block1[key] = np.asarray(refs_block1[key])
#     poss_block1[key] = np.asarray(poss_block1[key])
#     dec_vels_block1[key] = np.asarray(dec_vels_block1[key])
#     emgs_block1[key] = np.asarray(emgs_block1[key])
#     Ws_block1[key] = np.asarray(Ws_block1[key])
#     Hs_block1[key] = np.asarray(Hs_block1[key])
#     alphas_block1[key] = np.asarray(alphas_block1[key])
#     times_block1[key] = np.asarray(times_block1[key])
#     conditions_block1[key] = np.asarray(conditions_block1[key])
#     int_vel_block1[key] = np.asarray(int_vel_block1[key])
    
# GET BLOCK 2
# print('analyzing block 2')
# for key in keys:
#     print('analyzing', key)
#     PATH = 'data/' + key + '/BLOCK2/weiner_task_data*'
#     print(PATH)
#     data_dir = 'data/' + key + '/BLOCK2/'
#     for name in glob.glob(PATH):
#         file_name = os.path.basename(name)
#         print(file_name)
#         weiner_task_data, weiner_config = aopy.data.load_bmi3d_hdf_table(data_dir, file_name, 'weiner')
#         f = weiner_task_data
# #         print(f.dtype.names) # get the names of the fields
#         refs_block2[key].append(np.asarray(f['reference']))
#         poss_block2[key].append(np.asarray(f['decoded_position']))
#         dec_vels_block2[key].append(np.asarray(f['decoded_velocity']))
#         int_vel_block2[key] = np.asarray(f['intended_velocity']) # didn't seem to be saving
#         emgs_block2[key].append(np.asarray(f['filtered_emg']))
#         Ws_block2[key].append(np.asarray(f['weiner_filter_w']))
#         Hs_block2[key].append(np.asarray(f['weiner_filter_h'][0]))
#         alphas_block2[key].append(weiner_config['alpha'])
#         time = np.asarray(f['timestamp']).flatten()
#         times_block2[key].append(time - time[0])
#         conditions_block2[key].append(file_name[-13:-3])
#     refs_block2[key] = np.asarray(refs_block2[key])
#     poss_block2[key] = np.asarray(poss_block2[key])
#     dec_vels_block2[key] = np.asarray(dec_vels_block2[key])
#     emgs_block2[key] = np.asarray(emgs_block2[key])
#     Ws_block2[key] = np.asarray(Ws_block2[key])
#     Hs_block2[key] = np.asarray(Hs_block2[key])
#     alphas_block2[key] = np.asarray(alphas_block2[key])
#     times_block2[key] = np.asarray(times_block2[key])
#     conditions_block2[key] = np.asarray(conditions_block2[key])

# # GET BLOCK 3 (10 MINUTE LONG BLOCK)
# print('analyzing 10 minute')
# for key in keys:
#     print('analyzing', key)
#     PATH = 'data/' + key + '/10MIN/weiner_task_data*'
#     print(PATH)
#     data_dir = 'data/' + key + '/10MIN/'
#     for name in glob.glob(PATH):
#         file_name = os.path.basename(name)
#         print(file_name)
#         weiner_task_data, weiner_config = aopy.data.load_bmi3d_hdf_table(data_dir, file_name, 'weiner')
#         f = weiner_task_data
# #         print(f.dtype.names) # get the names of the fields
#         refs_block3[key].append(np.asarray(f['reference']))
#         poss_block3[key].append(np.asarray(f['decoded_position']))
#         dec_vels_block3[key].append(np.asarray(f['decoded_velocity']))
#         int_vel_block3[key] = np.asarray(f['intended_velocity']) # didn't seem to be saving
#         emgs_block3[key].append(np.asarray(f['filtered_emg']))
#         Ws_block3[key].append(np.asarray(f['weiner_filter_w']))
#         Hs_block3[key].append(np.asarray(f['weiner_filter_h'][0]))
#         alphas_block3[key].append(weiner_config['alpha'])
#         time = np.asarray(f['timestamp']).flatten()
#         times_block3[key].append(time - time[0])
#         conditions_block3[key].append(file_name[-13:-3])
#     refs_block3[key] = np.asarray(refs_block3[key])
#     poss_block3[key] = np.asarray(poss_block3[key])
#     dec_vels_block3[key] = np.asarray(dec_vels_block3[key])
#     emgs_block3[key] = np.asarray(emgs_block3[key])
#     Ws_block3[key] = np.asarray(Ws_block3[key])
#     Hs_block3[key] = np.asarray(Hs_block3[key])
#     alphas_block3[key] = np.asarray(alphas_block3[key])
#     times_block3[key] = np.asarray(times_block3[key])
#     conditions_block3[key] = np.asarray(conditions_block3[key])

In [ ]:
# # SAVE DATA
# data = [refs_block1,poss_block1,dec_vels_block1,int_vel_block1,emgs_block1,Ws_block1,Hs_block1,alphas_block1,times_block1,conditions_block1]

# with open('sprint3_data_block1.pickle','wb') as handle:
#     pickle.dump(data,handle,protocol=pickle.HIGHEST_PROTOCOL)

In [ ]:
# SAVE DATA
# data = [refs_block2,poss_block2,dec_vels_block2,int_vel_block2,emgs_block2,Ws_block2,Hs_block2,alphas_block2,times_block2,conditions_block2]

# with open('sprint3_data_block2.pickle','wb') as handle:
#     pickle.dump(data,handle,protocol=pickle.HIGHEST_PROTOCOL)

In [ ]:
data = [refs_block3,poss_block3,dec_vels_block3,int_vel_block3,emgs_block3,Ws_block3,Hs_block3,alphas_block3,times_block3,conditions_block3]

with open('sprint3_data_block3.pickle','wb') as handle:
    pickle.dump(data,handle,protocol=pickle.HIGHEST_PROTOCOL)

## saving alphaD vs alphaF data

In [ ]:
# weiner_task_data_META_S9_L2_a_5_d_5_block_0_aE_1e-06_aD_0.001_aF_1e-06.h5
# weiner_task_data_META_S9_L2_a_5_d_5_block_0_aE_1e-06_aD_0.01_aF_1e-06.h5
# weiner_task_data_META_S9_L2_a_5_d_5_block_0_aE_1e-06_aD_0.1_aF_1e-06.h5
# weiner_task_data_META_S9_L2_a_5_d_5_block_0_aE_1e-06_aD_1e-05_aF_1e-06.h5
# weiner_task_data_META_S9_L2_a_5_d_5_block_0_aE_1e-06_aD_1e-06_aF_1e-06.h5

keys = ['aD_0.001_aF_1e-06']
#         'aD_0.01_aF_1e-06',
#         'aD_0.1_aF_1e-06',
#         'aD_1e-05_aF_1e-06',
#         'aD_1e-06_aF_1e-06']

# create dataframe
var_names = ['refs','poss','dec_vels','int_vels','emgs','Ws','Hs','alphas','times','conditions']

refs_block3 = {key: [] for key in keys}
poss_block3 = {key: [] for key in keys}
dec_vels_block3 = {key: [] for key in keys}
emgs_block3 = {key: [] for key in keys}
int_vel_block3 = {key: [] for key in keys}
Ws_block3 = {key: [] for key in keys}
Hs_block3 = {key: [] for key in keys}
alphas_block3 = {key: [] for key in keys}
times_block3 = {key: [] for key in keys}
conditions_block3 = {key: [] for key in keys}

In [ ]:
for key in keys:
    print('analyzing', key)
    data_dir = '/Volumes/My Passport/test_runs/20220217-new-electrodes-grid-search-mm'
    file_name = 'weiner_task_data_META_S9_L2_a_5_d_5_block_0_aE_1e-06_' + key + '.h5'
    print(file_name)
    weiner_task_data, weiner_config = aopy.data.load_bmi3d_hdf_table(data_dir, file_name, 'weiner')
    f = weiner_task_data
    c = weiner_config
#     print(f.dtype.names) # get the names of the fields
    
    # get the data
    refs_block3[key].append(np.asarray(f['reference']))
    poss_block3[key].append(np.asarray(f['decoded_position']))
    dec_vels_block3[key].append(np.asarray(f['decoded_velocity']))
    int_vel_block3[key] = np.asarray(f['intended_velocity']) 
    emgs_block3[key].append(np.asarray(f['filtered_emg']))
    Ws_block3[key].append(np.asarray(f['weiner_filter_w']))
    Hs_block3[key].append(np.asarray(f['weiner_filter_h'][0]))
    alphas_block3[key].append(c['alpha'])
    time = np.asarray(f['timestamp']).flatten()
    times_block3[key].append(time - time[0])
    conditions_block3[key].append(file_name[-13:-3])
    
    # turn it into an array
    refs_block3[key] = np.asarray(refs_block3[key])
    poss_block3[key] = np.asarray(poss_block3[key])
    dec_vels_block3[key] = np.asarray(dec_vels_block3[key])
    emgs_block3[key] = np.asarray(emgs_block3[key])
    Ws_block3[key] = np.asarray(Ws_block3[key])
    Hs_block3[key] = np.asarray(Hs_block3[key])
    alphas_block3[key] = np.asarray(alphas_block3[key])
    times_block3[key] = np.asarray(times_block3[key])
    conditions_block3[key] = np.asarray(conditions_block3[key])
    
    print("finished ", key)
    print("")

In [ ]:
PATH = '/Volumes/My Passport/20220217-new-electrodes-grid-search-mm'

data = [refs_block3,poss_block3,dec_vels_block3,int_vel_block3,emgs_block3,Ws_block3,Hs_block3,alphas_block3,times_block3,conditions_block3]

with open(PATH + '20220217-new-electrodes-grid-search.pickle','wb') as handle:
    pickle.dump(data,handle,protocol=pickle.HIGHEST_PROTOCOL)

In [ ]:
c

In [ ]:
subject_number = 101
save_learning = 50
decoder_number = 1
alphaE = 1e-6
alphaD = 1e-4
alphaF = 1e-7
block_number = 1

In [ ]:
hdf_save_filename  = 'quattro_data_META_S' + str(subject_number) \
                    + '_L2_a_' + str(save_learning) + '_d_' + str(decoder_number) \
                    + '_block_' + str(block_number) + '_pE_' + '%.0e' %alphaE \
                    + '_pD_' + '%.0e' %alphaD + '_pF_' + '%.0e' %alphaF + '.h5'

In [ ]:
hdf_save_filename